In [19]:
from utils import *

import numpy as np
import torch
import matplotlib.pyplot as plt
import pandas as pd
import re


%matplotlib inline
%load_ext autoreload
%autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Sub-task A - Offensive language identification
Given a text, your system should categorise it into OFF (offensive) and NOT (not offensive).

## Load Data

In [18]:
path = '../data/Test A Release/testset-taska.tsv'

data_A = pd.read_table(path,index_col='id')
data_A.head()

,tweet
id,
15923,#WhoIsQ #WheresTheServer #DumpNike #DECLASFISA...
27014,"#ConstitutionDay is revered by Conservatives, ..."
30530,#FOXNews #NRA #MAGA #POTUS #TRUMP #2ndAmendmen...
13876,#Watching #Boomer getting the news that she is...
60133,#NoPasaran: Unity demo to oppose the far-right...


## Data Transformation and Cleaning

Work on the text of each tweet : Extracts : 
    - Hashtags
    - Words
    - Words cut on CAPs and Numbers 

In [142]:
data_A['hashtags'] = data_A['tweet'].apply(lambda x : re.compile('#(\w+)').findall(x))
data_A['words'] = data_A['tweet'].apply(lambda x : re.compile('\w+').findall(x))
data_A['words_refined'] = data_A['tweet'].apply(lambda x : re.compile('[0-9]+|[A-Z][a-z]+|[A-Z]+|[a-z]+').findall(x))
data_A.head(10)

,tweet,hashtags,words,words_refined
id,,,,
15923,#WhoIsQ #WheresTheServer #DumpNike #DECLASFISA...,"[WhoIsQ, WheresTheServer, DumpNike, DECLASFISA...","[WhoIsQ, WheresTheServer, DumpNike, DECLASFISA...","[Who, Is, Q, Wheres, The, Server, Dump, Nike, ..."
27014,"#ConstitutionDay is revered by Conservatives, ...",[ConstitutionDay],"[ConstitutionDay, is, revered, by, Conservativ...","[Constitution, Day, is, revered, by, Conservat..."
30530,#FOXNews #NRA #MAGA #POTUS #TRUMP #2ndAmendmen...,"[FOXNews, NRA, MAGA, POTUS, TRUMP, 2ndAmendmen...","[FOXNews, NRA, MAGA, POTUS, TRUMP, 2ndAmendmen...","[FOXN, ews, NRA, MAGA, POTUS, TRUMP, 2, nd, Am..."
13876,#Watching #Boomer getting the news that she is...,"[Watching, Boomer, Wentworth]","[Watching, Boomer, getting, the, news, that, s...","[Watching, Boomer, getting, the, news, that, s..."
60133,#NoPasaran: Unity demo to oppose the far-right...,"[NoPasaran, London, antifa, Oct13]","[NoPasaran, Unity, demo, to, oppose, the, far,...","[No, Pasaran, Unity, demo, to, oppose, the, fa..."
83681,. . . What the fuck did he do this time?,[],"[What, the, fuck, did, he, do, this, time]","[What, the, fuck, did, he, do, this, time]"
96874,#RAP is a form of ART! Used to express yoursel...,"[RAP, QUEEN]","[RAP, is, a, form, of, ART, Used, to, express,...","[RAP, is, a, form, of, ART, Used, to, express,..."
65507,@USER Do you get the feeling he is kissing @US...,[],"[USER, Do, you, get, the, feeling, he, is, kis...","[USER, Do, you, get, the, feeling, he, is, kis..."
78910,5 Tips to Enhance Audience Connection on Faceb...,"[socialmedia, smm]","[5, Tips, to, Enhance, Audience, Connection, o...","[5, Tips, to, Enhance, Audience, Connection, o..."
